In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

In [6]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME') # 步长为1，补零


def max_pool_2x2(x):
    return tf.nn.max_pool2d(x, ksize=[1,2,2,1],strides=[1,2,2,1], padding='SAME') # 2x2池化 步长为2 补零


def weight_variable(shape):
    init = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init)


def bias_variable(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)

def cnn(x):
    # with tf.name_scope('reshape'):
    x_image = tf.reshape(x, [-1,28,28,1])     # 不定数个 28x28x1 图片
    # 第一层卷积
    w1 = weight_variable([5,5,1,32]) # 32个 5x5x1 卷积核
    b1 = bias_variable([32])
    conv1 = tf.nn.relu(conv2d(x_image,w1) + b1) # 1个 28x28x32
    pool1 = max_pool_2x2(conv1)                 # 1个 14x14x32
    
    # 第二层卷积
    w2 = weight_variable([5,5,32,64]) # 64个 5x5x32 卷积核
    b2 = bias_variable([64])
    conv2 = tf.nn.relu(conv2d(pool1, w2) + b2) # 1个 14x14x64
    pool2 = max_pool_2x2(conv2)                # 1个 7x7x64
    
    # 全连接网络
    wfc1 = weight_variable([7*7*64, 1024])
    bfc1 = bias_variable([1024])
    
    # 把卷积网络中的4维图像转为二维图像
    p2 = tf.reshape(pool2, [-1, 7*7*64])
    fc1 = tf.nn.relu(tf.matmul(p2,wfc1) + bfc1)
    
    # 防止过拟合
    keep_prob = tf.placeholder(tf.float32)
    dfc1 = tf.nn.dropout(fc1, keep_prob)

    # 输出层
    wfc2 = weight_variable([1024,10])
    bfc2 = bias_variable([10])
    out = tf.matmul(dfc1, wfc2) + bfc2
    return out, keep_prob

In [3]:
# Import data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

# Create the model
x = tf.placeholder(tf.float32, [None, 784])

# Define loss and optimizer
y_ = tf.placeholder(tf.float32, [None, 10])

W0723 18:45:32.852883  5536 deprecation.py:323] From <ipython-input-3-e546b10b2310>:2: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0723 18:45:32.859861  5536 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0723 18:45:32.861856  5536 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for 

Extracting MNIST_data\train-images-idx3-ubyte.gz


W0723 18:45:33.197957  5536 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0723 18:45:33.207934  5536 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.
W0723 18:45:33.296693  5536 deprecation.py:323] From D:\Program Files (x86)\Anaconda3\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be 

Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [7]:
# 构造网络计算图
out, keep_prob = cnn(x)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_, logits=out)
loss = tf.reduce_mean(loss)

train_step = tf.train.AdadeltaOptimizer(0.01).minimize(loss)
pred = tf.equal(tf.arg_max(y_,1), tf.arg_max(out,1))
pred = tf.cast(pred, tf.float32)
accuracy = tf.reduce_mean(pred)


W0723 18:47:12.742788  5536 deprecation.py:506] From <ipython-input-6-ff3060a59433>:44: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Tensor("MaxPool2d_1:0", shape=(?, 7, 7, 64), dtype=float32)


W0723 18:47:13.039000  5536 deprecation.py:323] From <ipython-input-7-595f34607bf8>:7: arg_max (from tensorflow.python.ops.gen_math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.math.argmax` instead


In [ ]:
# 定义每个批次的大小
batch_size = 100
# 计算一共有多少个批次
n_batch = mnist.train.num_examples // batch_size

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    # 训练21个周期
    for epoch in range(3):
        # n_batch ： 一共有多少个批次
        for batch in range(n_batch):
            # 保存batch_size张图片的数据与标签
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y_:batch_ys,keep_prob:0.5})

        # 用测试集的图片及标签求得准确率
        acc = sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1.0})
        print("Iter "+ str(epoch) + ", Texting acc " + str(acc))